<a href="https://colab.research.google.com/github/hansufsm/FisicaIA/blob/main/esquadrao_fisicaia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎓 Projeto: Agentes IA para Estudantes Universitários
---

## 💡 Proposta: Sistema Colaborativo de Resolução de Problemas Físicos (Esquadrão Física IA)
**🎯Objetivo**: Criar um sistema de múltiplos agentes que ajuda estudantes a resolver problemas complexos de física, especialmente em eletricidade e magnetismo, com visualização, explicação passo a passo e conexão com aplicações do

### Como Utilizar Este Notebook
O notebook é dividido em blocos de código chamados "células". Você deve executar cada célula de cima para baixo, clicando no ícone ▶️ (play) ao lado dela.

✅ Etapa 1: Instalar as bibliotecas

Clique no botão ▶️ da primeira célula para instalar os pacotes necessários.

✅ Etapa 2: Inserir a sua **API Key**

Ao rodar essa célula, o notebook vai pedir que você cole a sua API Key do Google AI Studio.

Ela é necessária para o agente funcionar.

Se você não tem ainda:

* Vá para Google AI Studio
* Crie sua chave (grátis).
* Copie e cole no campo solicitado.

⚠️ Importante: Nunca cole sua chave direto no código nem compartilhe ela. Este notebook foi feito para usar a chave de forma segura.

✅ Etapa 3: Início da Interação

Assim que a API estiver conectada:

Continue executando Clique no botão ▶️ das célular de códigos seguintes até chegar à célula [Principal](https://colab.research.google.com/drive/1BXuUflNOri4yG8IMimmcQ0tU-O3tX98V#scrollTo=mOsQgUG8SPx6).

Clique no botão ▶️ da célula [Principal](https://colab.research.google.com/drive/1BXuUflNOri4yG8IMimmcQ0tU-O3tX98V#scrollTo=mOsQgUG8SPx6.) para iniciar a conversa

---


In [4]:
# Esquadrão FisicaIA - Sistema Colaborativo de Resolução de Problemas Físicos
# Desenvolvido por Prof. Hans Rogério Zimermann
# Implementação de 4 agentes especializados usando Gemini API
# Projeto Desafio Imersão alura Maio 2025

# ===== CONFIGURAÇÃO INICIAL =====

# Instalação das bibliotecas necessárias
!pip install -q google-generativeai genai streamlit matplotlib numpy pillow plotly

import os
from google import genai
import json
import time
import numpy as np
import matplotlib.pyplot as plt
import base64
from io import BytesIO
import plotly.graph_objects as go
from PIL import Image
import re
import threading
import queue


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.1/831.1 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.36.0 which is incompatible.


In [5]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

client = genai.Client()

# Listando os Modelos

#for model in client.models.list():
#  print(model.name)

# Criar uma variável para armazenar o Modelo a ser usado
modelo = "models/gemini-2.0-flash"

In [6]:
# Instalar Framework ADK de agentes do Google ##################################

!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72

In [7]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [8]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [9]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [20]:
# Definição dos Agentes

################################################
# --- Agente 1: Interpretador de Problemas --- #
################################################
def agente_interpretador(topico):
  interpretador = Agent(
      name="agente_interpretador_de_problemas",
      model="gemini-2.0-flash",
      instruction="""
      Você é um Agente Interpretador de Problemas de Física especializado em analisar enunciados.
      Sua função é:
      1. Analisar o enunciado do problema
      2. Classificar o tipo de problema (eletrostática, circuitos, magnetismo, etc.)
      3. Identificar e Extrair todas as variáveis, constantes e unidades mencionadas
      4. Listar as equações relevantes para a solução
      5. Sugerir uma abordagem didática e metodológica para resolver o problema

      Use o formato latex quando envolver expressões matemáticas. Quando for o caso ou possível, jamais usando equações inline, sempre em nova linha..

      """,
      description="Interpretador de Problemas sobre o tópico indicado",
      tools=[google_search]
  )

  entrada_do_agente_interpretador = f"Tópico: {topico}\n"
  # Executa o agente
  interpretacao_do_problema = call_agent(interpretador, entrada_do_agente_interpretador)
  return interpretacao_do_problema

################################################
# --- Agente 2: Interpretador de Problemas --- #
################################################
# O Agente Solucionador desenvolve a solução matemática

def agente_solucionador(topico):
    solucionador = Agent(
        name="agente_solucionador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        # Definição do papel do solucionador
        instruction="""
        Você é um Agente Solucionador Matemático especializado em física.
        Sua função é:
        1. Desenvolver a solução matemática passo a passo
        2. Realizar todos os cálculos e manipulações algébricas necessárias
        3. Verificar as unidades e consistência dimensional
        4. Fornecer o resultado final com unidades corretas

        Use o formato latex quando envolver expressões matemáticas. Quando for o caso ou possível.
        """,
        description="Agente solucionador",
        tools=[google_search]
    )

    entrada_do_agente_solucionador = f"\n\nEnunciado: {topico}\n"
    # Executa o agente
    plano_de_solucao = call_agent(solucionador, entrada_do_agente_solucionador)
    return plano_de_solucao

################################################
# --- Agente 3: Visualizador de problemas  --- #
################################################
# O Agente Visualizador cria representações visuais

def agente_visualizador(topico):
    visualizador = Agent(
        name="agente_visualizador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        # Definição do papel do solucionador
        instruction="""
        Você é um Agente Especialista e Visualização de dados, especializado em criar representações visuais de diversos problemas de física.
        Use o formato latex quando envolver expressões matemáticas. Quando for o caso ou possível. Use linguagem profissional e didática.
        Sua função é:
        1. Criar representações visuais do problema (diagramas de circuitos, campos, etc.)
        2. Gerar gráficos interativos para visualizar relações entre variáveis
        3. Produzir animações simples para demonstrar conceitos dinâmicos
        4. Adaptar visualizações para diferentes estilos de aprendizagem
        5. Gerar código Python para criar diagramas, gráficos ou visualizações relevantes
        6. Adaptar as visualizações ao tipo específico de problema
        7. Garantir que as visualizações sejam claras e informativas

        Formate sua resposta com blocos de código Python executáveis usando matplotlib, plotly ou outras bibliotecas.
        Inclua também uma descrição do que cada visualização representa.

        Para cada visualização, forneça:
        1. Uma descrição do que está sendo visualizado
        2. O código Python completo para gerar a visualização
        3. Uma explicação de como interpretar a visualização
        """,
        description="Agente visualizador",
        tools=[google_search]
    )

    entrada_do_agente_visualizador = f"\nVisualização: {topico}\n"
    # Executa o agente
    plano_de_visualizacao = call_agent(visualizador, entrada_do_agente_visualizador)
    return plano_de_visualizacao

################################################
# --- Agente 3: Visualizador de problemas  --- #
################################################
# O Agente Contextualizador adiciona aplicações práticas

def agente_contextualizador(topico):
    contextualizador = Agent(
        name="agente_contextualizador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        # Definição do papel do solucionador
        instruction="""

Claro! Aqui estão algumas sugestões para aprimorar o prompt, tornando-o mais claro e eficaz:

Especificar Detalhes do Contexto:

Incluir exemplos de como as aplicações práticas podem ser ligadas a situações do dia a dia.
Links e Recursos de Qualidade:

Especificar critérios para a seleção de links e vídeos, como autoridade e clareza das fontes.
Formatar Saídas:

Detalhar como as informações devem ser organizadas, possivelmente com uma estrutura padronizada para cada tipo de resposta (análise, links, exercícios).
Feedback e Revisão:

Incluir um passo para revisão do conteúdo gerado, garantindo precisão e relevância.
Interatividade:

Considerar somar algum aspecto interativo, como opções para o estudante explorar diferentes níveis de dificuldade ou temas.
Reformulação do Prompt:

Você é um Agente Contextualizador de Problemas de Física especializado em analisar enunciados e adicionar aplicações práticas. Sua função é:

Análise e Contextualização:

Identificar a área da Física envolvida no problema.
Adicionar aplicações práticas e exemplos concretos do dia a dia para tornar o conceito mais tangível.
Recursos Educativos:

Sugerir pelo menos três links para aprofundamento, hospedados em universidades federais, com foco em qualidade e clareza.
Propor um vídeo do YouTube de qualidade que complemente o aprendizado, escolhendo fontes confiáveis e reconhecidas.
Exercícios Práticos:

Propor um exercício de nível médio e outro desafiador, com soluções comentadas quando possível para feedback imediato.
Apresentação e Claridade:

Utilizar LaTeX para expressões matemáticas, garantindo clareza e apresentação didática, jamais usando equações inline, sempre em nova linha.
Empregar emojis de forma moderada para destacar pontos cruciais e manter o engajamento.
        """,
        description="Agente contextualizador",
        tools=[google_search]
    )

    entrada_do_agente_contextualizador = f"\nContexto: {topico}\n"
    # Executa o agente
    plano_de_contexto = call_agent(contextualizador, entrada_do_agente_contextualizador)
    return plano_de_contexto


# Principal

In [25]:
display(to_markdown("### 🚀 Iniciando o Sistema Colaborativo de Resolução de Problemas Físicos com 4 Agentes (🤖 - Esquadrão Física IA)"))

# --- Obter o Tópico do Usuário ---
#\n Um circuito elétrico contém uma bateria de 12V e um resistor de 4 ohms. Qual é a corrente no circuito?
topico = input("❓ Por favor, digite o enuciado do problema de Física: \n Exemplo: Qual é o campo elétrico a 0.2 m de uma carga de 2 μC? ")

# Inserir lógica do sistema de agentes ################################################

if not topico:
  print("Por favor informe um enunciado!")
else:
  display(to_markdown("## 👏 Ótimo "))
  print(f"Vamos resolver juntos: - {topico}!")

# Dicas:
# Qual é o campo elétrico a 0.2 m de uma carga de 2 μC?
#

  # alimenta o Agente 1 - agente_interpretador() com o problema (topico)
  problema_fisica = agente_interpretador(topico)
  print("\n")
  display(to_markdown("*--- Resposta do **Agente 1**: (Interpretador) ---*"))
  display(to_markdown(problema_fisica))
  print("-----------------------------------------------")

  # alimenta o Agente 2 - agente_solucionador() com a resposta do Agente 1 (problema_fisica)
  plano_de_solucao = agente_solucionador(problema_fisica)
  print("\n")
  display(to_markdown("*--- Resposta do **Agente 2**: (Solucionador) ---*"))
  display(to_markdown(plano_de_solucao))

  # alimenta o Agente 3 - agente_visualizador() com a resposta do Agente 2 (plano_de_solucao)
  plano_de_visualizacao = agente_visualizador(plano_de_solucao)
  print("\n")
  display(to_markdown("*--- Resposta do **Agente 3**: (Visualizador) ---*"))
  display(to_markdown(plano_de_visualizacao))

  # alimenta o Agente 4 - agente_contextualizador() com a resposta do Agente 1 (problema_fisica)
  plano_de_contexto = agente_contextualizador(problema_fisica)
  print("\n")
  display(to_markdown("*--- Resposta do **Agente 4**: (contextualizador) ---*"))
  display(to_markdown("### 🌍 Contextualização e Aplicações:\n"))
  display(to_markdown(plano_de_contexto))



> ### 🚀 Iniciando o Sistema Colaborativo de Resolução de Problemas Físicos com 4 Agentes (🤖 - Esquadrão Física IA)

❓ Por favor, digite o enuciado do problema de Física: 
 Exemplo: Qual é o campo elétrico a 0.2 m de uma carga de 2 μC? display(to_markdown(plano_de_contexto))


> ## 👏 Ótimo 

Vamos resolver juntos: - display(to_markdown(plano_de_contexto))!




> *--- Resposta do **Agente 1**: (Interpretador) ---*

...

> Okay, para começar, preciso que você me apresente um problema de física específico que você gostaria que eu visualizasse. Por favor, forneça o problema com o máximo de detalhes possível, incluindo:
> 
> *   **A descrição detalhada do problema:** Qual é a situação física que você quer analisar? (Ex: movimento de um projétil, circuito elétrico, etc.)
> *   **Valores numéricos:** Quais são os valores das grandezas envolvidas? (Ex: velocidade inicial, resistência, massa, etc.)
> *   **Unidades:** Quais são as unidades de medida de cada grandeza? (Ex: metros, segundos, ohms, etc.)
> *   **O que você quer encontrar:** Qual é a incógnita que você precisa determinar? (Ex: alcance máximo, corrente elétrica, energia potencial, etc.)
> *   **Quais visualizações seriam mais úteis:** Você tem alguma ideia de que tipo de gráfico ou animação seria mais útil para entender o problema?
> 
> Com essas informações, posso criar visualizações claras e informativas para ajudá-lo a entender e resolver o problema.
> 


> *--- Resposta do **Agente 4**: (contextualizador) ---*

> ### 🌍 Contextualização e Aplicações:


> Perfeito! Vamos começar com este problema:
> 
> Um bloco de gelo de 2 kg, inicialmente a -10°C, recebe calor até se transformar completamente em água a 20°C. Calcule a quantidade total de calor que o bloco de gelo absorve durante este processo.
> 
> Dados:
> *   Calor específico do gelo: $c_{gelo} = 2100 \, \text{J/(kg} \cdot \text{°C)}$
> *   Calor latente de fusão do gelo: $L_f = 334 \times 10^3 \, \text{J/kg}$
> *   Calor específico da água: $c_{água} = 4186 \, \text{J/(kg} \cdot \text{°C)}$
> 
> Analise este problema, detalhando as etapas, fornecendo links úteis, um vídeo explicativo e exercícios adicionais para praticar. Estou ansioso para ver sua análise!
> 


Perfeito! Vamos começar com este problema:

Um bloco de gelo de 2 kg, inicialmente a -10°C, recebe calor até se transformar completamente em água a 20°C. Calcule a quantidade total de calor que o bloco de gelo absorve durante este processo.

Dados:
*   Calor específico do gelo: $c_{gelo} = 2100 \, \text{J/(kg} \cdot \text{°C)}$
*   Calor latente de fusão do gelo: $L_f = 334 \times 10^3 \, \text{J/kg}$
*   Calor específico da água: $c_{água} = 4186 \, \text{J/(kg} \cdot \text{°C)}$

Analise este problema, detalhando as etapas, fornecendo links úteis, um vídeo explicativo e exercícios adicionais para praticar. Estou ansioso para ver sua análise!




In [ ]:
 import matplotlib.pyplot as plt
 import numpy as np


 # Constantes
 k = 8.99e9  # N.m^2/C^2
 q = 2e-6  # C


 # Distâncias
 r = np.linspace(0.1, 1, 100)  # de 0.1m a 1m


 # Calcula o campo elétrico
 E = k * q / (r**2)


 # Cria o gráfico
 plt.figure(figsize=(10, 6))
 plt.plot(r, E, label='Campo Elétrico (E)')
 plt.xlabel('Distância (r) em metros')
 plt.ylabel('Campo Elétrico (E) em N/C')
 plt.title('Campo Elétrico vs. Distância de uma Carga Puntiforme')
 plt.grid(True)
 plt.legend()
 plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constantes
k = 8.9875e9  # N m^2/C^2
q = 2e-6      # C
r = 0.2       # m
E = 449375    # N/C (calculado)

# Criação do gráfico
fig, ax = plt.subplots(figsize=(8, 8))

# Desenha a carga no centro
ax.plot(0, 0, 'ro', markersize=10, label='Carga q')

# Cria vetores de campo elétrico
num_arrows = 20
for i in range(num_arrows):
    theta = 2 * np.pi * i / num_arrows
    x = r * np.cos(theta)
    y = r * np.sin(theta)

    # Normaliza o vetor para um tamanho visualmente agradável
    scale = 0.1
    dx = scale * np.cos(theta)
    dy = scale * np.sin(theta)

    ax.arrow(0, 0, x, y, head_width=0.05, head_length=0.1, fc='blue', ec='blue', alpha=0.5)

# Configurações do gráfico
ax.set_xlim(-0.5, 0.5)
ax.set_ylim(-0.5, 0.5)
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('Campo Elétrico ao Redor de uma Carga Pontual')
ax.grid(True)
ax.legend()

plt.show()